In [1]:
import numpy as np
import xarray as xr
from glob import glob
import cftime
import dask

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit="64G")
client = Client(cluster)
client

In [ ]:
for y in np.arange(1996, 2016):
    year = str(y)
    print(year)
    if year in ["2001", "2014", "2015"]:
        data_filesREF_gridU = [sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m0[1-9]d??.5d_gridU.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m1[0-1]d??.5d_gridU.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m12d[0-2]?.5d_gridU.nc"))][0]
        data_filesREF_gridUlast = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m12d31.5d_gridU.nc"))
        data_filesREF_gridV = [sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m0[1-9]d??.5d_gridV.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m1[0-1]d??.5d_gridV.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m12d[0-2]?.5d_gridV.nc"))][0]
        data_filesREF_gridVlast = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                              + year + "/CREG12.L75-REF08_y????m12d31.5d_gridV.nc"))
        UREF_base = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridU, coords="minimal", compat="override", 
                                                   parallel=True, decode_cf=False))
        VREF_base = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridV, coords="minimal", compat="override", 
                                                   parallel=True, decode_cf=False))
        UREFadd = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridUlast, coords="minimal", compat="override", 
                                                  parallel=True, decode_cf=False))
        VREFadd = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridVlast, coords="minimal", compat="override", 
                                                  parallel=True, decode_cf=False))
        UREFadd["time_counter"] = xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                  end=year + "-12-31 12:00:00", 
                                                  freq="5D", calendar="noleap")
        UREFadd["time_centered"].data = np.array(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                                 end=year + "-12-31 12:00:00", 
                                                                 freq="5D", calendar="noleap"))
        UREFadd["time_counter_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                       freq="3D", calendar="noleap"))[None, :]
        UREFadd["time_centered_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                        freq="3D", calendar="noleap"))[None, :]
        VREFadd["time_counter"] = xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                  end=year + "-12-31 12:00:00", 
                                                  freq="5D", calendar="noleap")
        VREFadd["time_centered"].data = np.array(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                                 end=year + "-12-31 12:00:00", 
                                                                 freq="5D", calendar="noleap"))
        VREFadd["time_counter_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                       freq="3D", calendar="noleap"))[None, :]
        VREFadd["time_centered_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                        freq="3D", calendar="noleap"))[None, :]
        UREFadd["nav_lat"] = UREFadd.nav_lat.expand_dims("time_counter")
        UREFadd["nav_lon"] = UREFadd.nav_lon.expand_dims("time_counter")
        VREFadd["nav_lat"] = VREFadd.nav_lat.expand_dims("time_counter")
        VREFadd["nav_lon"] = VREFadd.nav_lon.expand_dims("time_counter")
        uUREF_base = UREF_base.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        vVREF_base = VREF_base.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        uUREFadd = UREFadd.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        vVREFadd = VREFadd.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        dudzREF_base = uUREF_base.differentiate("deptht")
        dvdzREF_base = vVREF_base.differentiate("deptht")
        shearREF_base = ((dudzREF_base**2) + (dvdzREF_base**2)).rename("vovshear").chunk({"time_counter": 1})
        dudzREFadd = uUREFadd.differentiate("deptht")
        dvdzREFadd = vVREFadd.differentiate("deptht")
        shearREFadd = ((dudzREFadd**2) + (dvdzREFadd**2)).rename("vovshear").chunk({"time_counter": 1})
        shearREF = xr.concat([shearREF_base, shearREFadd], dim="time_counter", 
                     coords="minimal", compat="override").to_dataset(name="vovshear")
        days, datasets = zip(*shearREF.groupby("time_counter"))
        datasets = list(datasets)
        datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
        dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
        dates = dates[0:36] + list(xr.cftime_range(start=str(days[0].year) 
                                                   + "-07-05", end=str(days[0].year) 
                                                   + "-12-31", freq="5D", calendar="noleap"))
        dates = dates[:] + list(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                end=year + "-12-31 12:00:00", 
                                                freq="5D", calendar="noleap"))
    else:
        data_filesREF_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                          + year + "/CREG12.L75-REF08_*.5d_gridU.nc"))
        data_filesREF_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08/" 
                                          + year + "/CREG12.L75-REF08_*.5d_gridV.nc"))
        UREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridU, coords="minimal", compat="override", 
                                              parallel=True, decode_cf=False))
        VREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridV, coords="minimal", compat="override", 
                                              parallel=True, decode_cf=False))
        uUREF = UREF.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)})
        vVREF = VREF.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)})
        dudzREF = uUREF.differentiate("deptht")
        dvdzREF = vVREF.differentiate("deptht")
        shearREF = ((dudzREF**2) + (dvdzREF**2)).to_dataset(name="vovshear")
        days, datasets = zip(*shearREF.groupby("time_counter"))
        datasets = list(datasets)
        datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
        dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-shear/" + year + "/CREG12.L75-REF08_y" 
                 + "%04d" % d.year + "m" + "%02d" % d.month + "d" + "%02d" % d.day + ".5d_shear.nc") for d in dates]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")

1996


In [ ]:
print("clim")
data_filesREF_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m*_gridUclim.nc"))
data_filesREF_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m*_gridVclim.nc"))
UREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
VREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
uUREF = UREF.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
vVREF = VREF.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
dudzREF = uUREF.differentiate("deptht").chunk({"time_counter": 1})
dvdzREF = vVREF.differentiate("deptht").chunk({"time_counter": 1})
shearREF = ((dudzREF**2) + (dvdzREF**2)).to_dataset(name="vovshear")
path = str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-shear/clim/CREG12.L75-REF08_1996-2015_climatology_vovshear.nc")
shearREF.to_netcdf(path, engine="netcdf4", format="NETCDF4")

In [ ]:
for y in np.arange(2051, 2071):
    year = str(y)
    print(year)
    if year in ["2063"]:
        data_filesFUT_gridU = [sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m0[1-9]d??.5d_gridU.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m1[0-1]d??.5d_gridU.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m12d[0-2]?.5d_gridU.nc"))][0]
        data_filesFUT_gridUlast = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m12d31.5d_gridU.nc"))
        data_filesFUT_gridV = [sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m0[1-9]d??.5d_gridV.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m1[0-1]d??.5d_gridV.nc"))
                              + sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m12d[0-2]?.5d_gridV.nc"))][0]
        data_filesFUT_gridVlast = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                              + year + "/CREG12.L75-FUT08_y????m12d31.5d_gridV.nc"))
        UFUT_base = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridU, coords="minimal", compat="override", 
                                                   parallel=True, decode_cf=False))
        VFUT_base = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridV, coords="minimal", compat="override", 
                                                   parallel=True, decode_cf=False))
        UFUTadd = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridUlast, coords="minimal", compat="override", 
                                                  parallel=True, decode_cf=False))
        VFUTadd = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridVlast, coords="minimal", compat="override", 
                                                  parallel=True, decode_cf=False))
        UFUTadd["time_counter"] = xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                  end=year + "-12-31 12:00:00", 
                                                  freq="5D", calendar="noleap")
        UFUTadd["time_centered"].data = np.array(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                                 end=year + "-12-31 12:00:00", 
                                                                 freq="5D", calendar="noleap"))
        UFUTadd["time_counter_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                       freq="3D", calendar="noleap"))[None, :]
        UFUTadd["time_centered_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                        freq="3D", calendar="noleap"))[None, :]
        VFUTadd["time_counter"] = xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                  end=year + "-12-31 12:00:00", 
                                                  freq="5D", calendar="noleap")
        VFUTadd["time_centered"].data = np.array(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                                 end=year + "-12-31 12:00:00", 
                                                                 freq="5D", calendar="noleap"))
        VFUTadd["time_counter_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                       freq="3D", calendar="noleap"))[None, :]
        VFUTadd["time_centered_bounds"].data = np.array(xr.cftime_range(start=year + "-12-28 12:00:00", 
                                                                       end=year + "-12-31 12:00:00", 
                                                                        freq="3D", calendar="noleap"))[None, :]
        UFUTadd["nav_lat"] = UFUTadd.nav_lat.expand_dims("time_counter")
        UFUTadd["nav_lon"] = UFUTadd.nav_lon.expand_dims("time_counter")
        VFUTadd["nav_lat"] = VFUTadd.nav_lat.expand_dims("time_counter")
        VFUTadd["nav_lon"] = VFUTadd.nav_lon.expand_dims("time_counter")
        uUFUT_base = UFUT_base.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        vVFUT_base = VFUT_base.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        uUFUTadd = UFUTadd.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        vVFUTadd = VFUTadd.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)}).chunk({"time_counter": 1})
        dudzFUT_base = uUFUT_base.differentiate("deptht")
        dvdzFUT_base = vVFUT_base.differentiate("deptht")
        shearFUT_base = ((dudzFUT_base**2) + (dvdzFUT_base**2)).rename("vovshear").chunk({"time_counter": 1})
        dudzFUTadd = uUFUTadd.differentiate("deptht")
        dvdzFUTadd = vVFUTadd.differentiate("deptht")
        shearFUTadd = ((dudzFUTadd**2) + (dvdzFUTadd**2)).rename("vovshear").chunk({"time_counter": 1})
        shearFUT = xr.concat([shearFUT_base, shearFUTadd], dim="time_counter", 
                     coords="minimal", compat="override").to_dataset(name="vovshear")
        days, datasets = zip(*shearFUT.groupby("time_counter"))
        datasets = list(datasets)
        datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
        dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
        dates = dates[0:36] + list(xr.cftime_range(start=str(days[0].year) 
                                                   + "-07-05", end=str(days[0].year) 
                                                   + "-12-31", freq="5D", calendar="noleap"))
        dates = dates[:] + list(xr.cftime_range(start=year + "-12-31 12:00:00", 
                                                end=year + "-12-31 12:00:00", 
                                                freq="5D", calendar="noleap"))
    else:
        data_filesFUT_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                          + year + "/CREG12.L75-FUT08_*.5d_gridU.nc"))
        data_filesFUT_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08/" 
                                          + year + "/CREG12.L75-FUT08_*.5d_gridV.nc"))
        UFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridU, coords="minimal", compat="override", 
                                              parallel=True, decode_cf=False))
        VFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridV, coords="minimal", compat="override", 
                                              parallel=True, decode_cf=False))
        uUFUT = UFUT.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)})
        vVFUT = VFUT.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({
            "x": np.arange(0, 1580), "y": np.arange(0, 1801)})
        dudzFUT = uUFUT.differentiate("deptht")
        dvdzFUT = vVFUT.differentiate("deptht")
        shearFUT = ((dudzFUT**2) + (dvdzFUT**2)).to_dataset(name="vovshear")
        days, datasets = zip(*shearFUT.groupby("time_counter"))
        datasets = list(datasets)
        datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
        dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-shear/" + year + "/CREG12.L75-FUT08_y" 
                 + "%04d" % d.year + "m" + "%02d" % d.month + "d" + "%02d" % d.day + ".5d_shear.nc") for d in dates]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")

In [ ]:
print("clim")
data_filesFUT_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m*_gridUclim.nc"))
data_filesFUT_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m*_gridVclim.nc"))
UFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
VFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
uUFUT = UFUT.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
vVFUT = VFUT.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
dudzFUT = uUFUT.differentiate("deptht").chunk({"time_counter": 1})
dvdzFUT = vVFUT.differentiate("deptht").chunk({"time_counter": 1})
shearFUT = ((dudzFUT**2) + (dvdzFUT**2)).to_dataset(name="vovshear")
path = str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-shear/clim/CREG12.L75-FUT08_2051-2070_climatology_vovshear.nc")
shearFUT.to_netcdf(path, engine="netcdf4", format="NETCDF4")